# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint
import scipy.stats as st

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
loadPath = '../WeatherPy/weather_data.csv'

weatherFrame = pd.read_csv(loadPath)

print(weatherFrame.head())

           City    Lat     Lon  Max Temp  Humidity  Cloudiness  Wind Speed  \
0         Pisco -13.70  -76.22     63.00        82           0       13.87   
1        Lompoc  34.64 -120.46     64.99        52           1        6.85   
2        Faanui -16.48 -151.75     78.46        80           4       11.86   
3  Norman Wells  65.28 -126.83     23.00        67          75        3.36   
4  Ponta do Sol  32.67  -17.10     69.80        73          20       16.11   

  County        Date  
0     PE  1602560253  
1     US  1602560096  
2     PF  1602560254  
3     CA  1602560254  
4     PT  1602560254  


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

humidityFrame = weatherFrame[['Lat', 'Lon', 'Humidity']]

locations = humidityFrame[['Lat', 'Lon']]
weights = humidityFrame['Humidity']

print(humidityFrame.head())

     Lat     Lon  Humidity
0 -13.70  -76.22        82
1  34.64 -120.46        52
2 -16.48 -151.75        80
3  65.28 -126.83        67
4  32.67  -17.10        73


In [5]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowedWeatherData = weatherFrame.loc[weatherFrame['Humidity'] < 40 & weatherFrame['Cloudiness'] < 40]

print(narrowedWeatherData.head())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
